# **Amazon Lookout for Equipment** - 익명화한 익스펜더 데이터셋에 대한 데모
*파트 5: 정기적인 추론 호출 스케줄링*

In [1]:
BUCKET = '<YOUR_BUCKET_NAME_HERE>'
PREFIX = 'data/scheduled_inference'

## 초기화
---
이 노트북에서는 데이터 폴더에 추론 디렉토리를 추가하게끔 저장소 구조를 갱신합니다.
```
/lookout-equipment-demo
|
+-- data/
|   |
|   +-- inference/
|   |   |
|   |   |-- input/
|   |   |
|   |   \-- output/
|   |
|   +-- labelled-data/
|   |   \-- labels.csv
|   |
|   \-- training-data/
|       \-- expander/
|           |-- subsystem-01
|           |   \-- subsystem-01.csv
|           |
|           |-- subsystem-02
|           |   \-- subsystem-02.csv
|           |
|           |-- ...
|           |
|           \-- subsystem-24
|               \-- subsystem-24.csv
|
+-- dataset/
|   |-- labels.csv
|   |-- tags_description.csv
|   |-- timeranges.txt
|   \-- timeseries.zip
|
+-- notebooks/
|   |-- 1_data_preparation.ipynb
|   |-- 2_dataset_creation.ipynb
|   |-- 3_model_training.ipynb
|   |-- 4_model_evaluation.ipynb
|   \-- 5_inference_scheduling.ipynb        <<< 본 노트북 <<<
|
+-- utils/
    |-- lookout_equipment_utils.py
    \-- lookoutequipment.json
```

### 임포트

In [2]:
%%sh
pip -q install --upgrade pip
pip -q install --upgrade awscli boto3 sagemaker
aws configure add-model --service-model file://../utils/lookoutequipment.json --service-name lookoutequipment

In [3]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import boto3
import datetime
import os
import pandas as pd
import pprint
import pyarrow as pa
import pyarrow.parquet as pq
import sagemaker
import s3fs
import sys
import time
import uuid
import warnings

# Lookout for Equipment API 호출 관리를 위한 Helper 함수
sys.path.append('../utils')
import lookout_equipment_utils as lookout

### 파라미터

In [5]:
warnings.filterwarnings('ignore')

DATA       = os.path.join('..', 'data')
RAW_DATA   = os.path.join('..', 'dataset')
INFER_DATA = os.path.join(DATA, 'inference')


os.makedirs(os.path.join(INFER_DATA, 'input'), exist_ok=True)
os.makedirs(os.path.join(INFER_DATA, 'output'), exist_ok=True)

ROLE_ARN = sagemaker.get_execution_role()
REGION_NAME = boto3.session.Session().region_name

## 추론 스케줄러 생성하기
---
콘솔의 모델 세부 정보 부분으로 이동하면 추론 스케줄이 아직 없음을 확인할 수 있습니다.

![Schedule Starting point](../assets/schedule_start.png)

### 스케줄러 설정
새로운 추론 스케줄을 만들어 보겠습니다. 파라미터 일부는 필수 입력이지만 파라미터 다수는 유연하게 추가 설정할 수 있습니다.

#### 파라미터

* 추론을 위해 데이터를 업로드할 빈도로 `DATA_UPLOAD_FREQUENCY`를 설정합니다. 허용되는 값은`PT5M`,`PT10M`,`PT15M`,`PT30M`과`PT1H`입니다.
  * 이것은 추론 스케줄러가 실행되는 빈도와 데이터가 소스 버킷에 업로드되는 빈도입니다.
  * **참고** : ***업로드 빈도는 훈련 때 선택한 샘플링 비율과 호환되어야합니다.*** *예를 들어 모델을 30분 간격의 리샘플링으로 훈련시킨 경우 5분은 가능하지 않습니다. 추론 시 파라미터로 PT30M 또는 PT1H를 선택해야합니다.*
* 추론 데이터의 S3 버킷으로 `INFERENCE_DATA_SOURCE_BUCKET`를 설정합니다.
* 추론 데이터의 S3 접두사로 `INFERENCE_DATA_SOURCE_PREFIX`를 설정합니다.
* 추론 결과를 원하는 S3 버킷으로 `INFERENCE_DATA_OUTPUT_BUCKET`를 설정합니다.
* 추론 결과를 원하는 S3 접두사로 `INFERENCE_DATA_OUTPUT_PREFIX`를 설정합니다.
* 추론할 데이터를 **읽고** 추론 출력을 **쓸** 때 사용할 역할로 `ROLE_ARN_FOR_INFERENCE`를 설정합니다.

In [6]:
# 생성하려는 추론 스케줄러의 이름
INFERENCE_SCHEDULER_NAME = 'lookout-demo-model-v1-scheduler'

# 본 추론 스케줄러를 생성할 모델의 이름
MODEL_NAME_FOR_CREATING_INFERENCE_SCHEDULER = 'lookout-demo-model-v1'

# 필수 입력 파라미터
INFERENCE_DATA_SOURCE_BUCKET = BUCKET
INFERENCE_DATA_SOURCE_PREFIX = f'{PREFIX}/input/'
INFERENCE_DATA_OUTPUT_BUCKET = BUCKET
INFERENCE_DATA_OUTPUT_PREFIX = f'{PREFIX}/output/'
ROLE_ARN_FOR_INFERENCE = ROLE_ARN
DATA_UPLOAD_FREQUENCY = 'PT5M' 

#### 생략 가능한 파라미터

* 데이터 업로드하는데 지연이 예상되는 시간(분)으로 `DATA_DELAY_OFFSET_IN_MINUTES`를 설정합니다. 즉, 데이터 업로드하는 시간에 대한 버퍼입니다.
* ``INPUT_TIMEZONE_OFFSET``을 설정합니다. 허용되는 값은 +00:00, +00:30, -01:00, ... +11:30, +12:00, -00:00, -00:30, -01:00, ... -11:30, -12:00입니다.
* `TIMESTAMP_FORMAT`을 설정합니다. 허용되는 값은 `EPOCH`, `yyyy-MM-dd-HH-mm-ss` 또는 `yyyyMMddHHmmss`입니다. 이것은 입력 데이터 파일 명에 접미사로 붙는 타임스탬프 형식입니다. 이것은 Lookout Equipment에서 추론을 실행할 파일을 파악하는 데 사용됩니다 (그러므로 스케줄러가 실행할 파일을 찾게 하기 위해 이전 파일을 제거할 필요가 없음).
* `COMPONENT_TIMESTAMP_DELIMITER`를 설정합니다. 허용되는 값은 `-`, `_` 또는 ` `입니다. 입력 파일 명의 타임스탬프에서 구성 요소를 분리할 때 사용하는 구분자입니다.

In [7]:
DATA_DELAY_OFFSET_IN_MINUTES = None
INPUT_TIMEZONE_OFFSET = '+00:00'
COMPONENT_TIMESTAMP_DELIMITER = '_'
TIMESTAMP_FORMAT = 'yyyyMMddHHmmss'

### 추론 스케줄러 생성하기
CreateInferenceScheduler API는 스케줄러를 생성**하고** 구동시킵니다. 즉, 즉각적으로 비용이 발생하기 시작합니다. 그러나 기존 스케줄러를 원하는대로 중지하거나 재구동시킬 수 있습니다 (이 노트북의 마지막 부분 참조).

In [8]:
scheduler = lookout.LookoutEquipmentScheduler(
    scheduler_name=INFERENCE_SCHEDULER_NAME,
    model_name=MODEL_NAME_FOR_CREATING_INFERENCE_SCHEDULER,
    region_name=REGION_NAME
)

scheduler_params = {
    'input_bucket': INFERENCE_DATA_SOURCE_BUCKET,
    'input_prefix': INFERENCE_DATA_SOURCE_PREFIX,
    'output_bucket': INFERENCE_DATA_OUTPUT_BUCKET,
    'output_prefix': INFERENCE_DATA_OUTPUT_PREFIX,
    'role_arn': ROLE_ARN_FOR_INFERENCE,
    'upload_frequency': DATA_UPLOAD_FREQUENCY,
    'delay_offset': DATA_DELAY_OFFSET_IN_MINUTES,
    'timezone_offset': INPUT_TIMEZONE_OFFSET,
    'component_delimiter': COMPONENT_TIMESTAMP_DELIMITER,
    'timestamp_format': TIMESTAMP_FORMAT
}

scheduler.set_parameters(**scheduler_params)

## 추론 데이터 준비하기
---
스케줄러가 모니터링할 S3 입력 위치에 몇 가지 데이터를 준비하고 전송하겠습니다. 

In [9]:
# 원본 신호 전체를 불러오겠습니다.
all_tags_fname = os.path.join(DATA, 'training-data', 'expander.parquet')
table = pq.read_table(all_tags_fname)
all_tags_df = table.to_pandas()
del table
all_tags_df.head()

,signal-001,signal-002,signal-003,signal-004,signal-005,signal-006,signal-007,signal-008,signal-009,signal-010,...,signal-113,signal-114,signal-115,signal-116,signal-117,signal-118,signal-119,signal-120,signal-121,signal-122
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0.392371,0.545005,0.296774,0.413289,0.170744,0.482980,0.222063,0.268691,0.749860,0.475116,...,0.939024,0.830769,0.811321,0.653465,0.789474,0.810345,0.803571,0.787879,0.764706,0.810345
2015-01-01 00:01:00,0.389415,0.569155,0.290645,0.415646,0.142368,0.532297,0.222063,0.290804,0.776781,0.486884,...,0.939024,0.830769,0.811321,0.653465,0.789474,0.810345,0.803571,0.787879,0.779412,0.810345
2015-01-01 00:02:00,0.378179,0.547750,0.290645,0.406456,0.160959,0.470115,0.235673,0.277115,0.782389,0.472665,...,0.939024,0.830769,0.811321,0.653465,0.789474,0.810345,0.803571,0.787879,0.764706,0.810345
2015-01-01 00:03:00,0.381135,0.547750,0.284516,0.401744,0.170744,0.498794,0.249284,0.270446,0.771733,0.484432,...,0.939024,0.830769,0.811321,0.653465,0.789474,0.810345,0.803571,0.787879,0.764706,0.810345
2015-01-01 00:04:00,0.381135,0.553238,0.284516,0.406456,0.142368,0.493433,0.194842,0.272025,0.749860,0.484432,...,0.939024,0.830769,0.811321,0.653465,0.789474,0.810345,0.803571,0.787879,0.764706,0.810345


태그 설명을 불러옵시다. 본 데이터셋에는 다음 내용을 포함하는 태그 설명 파일이 존재합니다.

* `Tag`: 이력 관리 시스템에 고객이 기록한 태그 명 (예컨대 [Honeywell 프로세스 이력 데이터베이스](https://www.honeywellprocess.com/en-US/explore/products/advanced-applications/uniformance/Pages/uniformance-phd.aspx))
* `UOM`: 기록한 신호의 측정 단위
* `Subsystem`: 해당 센서가 연결된 자산 부속의 ID

여기에서 구성 요소 (즉, 하위 시스템 열)의 List를 수집할 수 있습니다. 

In [10]:
tags_description_fname = os.path.join(RAW_DATA, 'tags_description.csv')
tags_description_df = pd.read_csv(tags_description_fname)
components = tags_description_df['Subsystem'].unique()
tags_description_df.head()

,Tag,UOM,Subsystem
0,signal-001,micra pp,subsystem-05
1,signal-002,micra pp,subsystem-05
2,signal-003,micra pp,subsystem-05
3,signal-004,micra pp,subsystem-05
4,signal-005,micra pp,subsystem-08


샘플 추론 데이터셋을 구성하기 위해 원본 시계열 검증 기간에서 마지막 몇 분을 추출합니다. 

In [11]:
# 추출하려는 시퀀스 개수
num_sequences = 3

# 스케줄링 빈도 (분): 이 값은 **반드시**
# 모델 학습에 사용한 리샘플링 비율에 맞춰 설정해야 합니다. 
frequency = 5

# 각 시퀀스를 반복합니다.
start = all_tags_df.index.max() + datetime.timedelta(minutes=-frequency * (num_sequences) + 1)
for i in range(num_sequences):
    end = start + datetime.timedelta(minutes=+frequency - 1)
    
    # 이전 5분 단위로 시간을 반올림합니다.
    tm = datetime.datetime.now()
    tm = tm - datetime.timedelta(
        minutes=tm.minute % frequency,
        seconds=tm.second,
        microseconds=tm.microsecond
    )
    tm = tm + datetime.timedelta(minutes=+frequency * (i))
    tm = tm - datetime.timedelta(hours=9) # KST에 따른 조정
    current_timestamp = (tm).strftime(format='%Y%m%d%H%M%S')

    # 각 시퀀스마다 구성 요소 전체를 반복합니다. 
    print(f'Extracting data from {start} to {end}:')
    new_index = None
    for component in components:
        # 해당 구성 요소와 특정 시간 범위에 대해 Dataframe을 추출합니다.
        signals = list(tags_description_df.loc[(tags_description_df['Subsystem'] == component), 'Tag'])
        signals_df = all_tags_df.loc[start:end, signals]
        
        # 스케줄러가 추론을 실행할 시간에 맞게끔
        # 인덱스를 재설정해야 합니다. 
        if new_index is None:
            new_index = pd.date_range(
                start=tm,
                periods=signals_df.shape[0], 
                freq='1min'
            )
        signals_df.index = new_index
        signals_df.index.name = 'Timestamp'
        signals_df = signals_df.reset_index()
        signals_df['Timestamp'] = signals_df['Timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f')

        # 해당 파일을 CSV 형식으로 내보냅니다. 
        component_fname = os.path.join(INFER_DATA, 'input', f'{component}_{current_timestamp}.csv')
        signals_df.to_csv(component_fname, index=None)
    
    start = start + datetime.timedelta(minutes=+frequency)
    
# 입력 위치의 전체 폴더를 S3에 업로드합니다. 
INFERENCE_INPUT = os.path.join(INFER_DATA, 'input')
!aws s3 cp --recursive --quiet $INFERENCE_INPUT s3://$BUCKET/$PREFIX/input
    
# 이제 데이터를 준비했으므로 다음을 실행하여 스케줄러를 만듭니다.
create_scheduler_response = scheduler.create()

Extracting data from 2015-11-30 23:45:00 to 2015-11-30 23:49:00:
Extracting data from 2015-11-30 23:50:00 to 2015-11-30 23:54:00:
Extracting data from 2015-11-30 23:55:00 to 2015-11-30 23:59:00:
===== Polling Inference Scheduler Status =====

Scheduler Status: PENDING
Scheduler Status: RUNNING

===== End of Polling Inference Scheduler Status =====


스케줄러가 실행 중이며 추론 기록은 현재 비어 있습니다.

![Scheduler created](../assets/schedule_created.png)

## 추론 결과 얻기
---

### 추론 실행 결과 나열하기

**스케줄러가 추론을 최초로 실행할 경우 5-15분 정도 걸립니다.** 대기가 끝나면 현재 추론 스케줄러에서 ListInferenceExecution API를 사용할 수 있습니다. 입력 파라미터로 스케줄러 명만 필요합니다.

추론 실행 결과를 질의할 기간을 선택할 수 있습니다. 지정하지 않으면 추론 스케줄러에 대한 모든 실행 결과들이 나열됩니다. 시간 범위를 지정하려면 다음과 같이 합니다.

```python
START_TIME_FOR_INFERENCE_EXECUTIONS = datetime.datetime(2010,1,3,0,0,0)
END_TIME_FOR_INFERENCE_EXECUTIONS = datetime.datetime(2010,1,5,0,0,0)
```

즉, `2010-01-03 00:00:00`부터 `2010-01-05 00:00:00`까지의 실행 결과들이 나열됩니다.

특정 상태의 실행 결과를 질의하도록 선택할 수도 있습니다. 허용되는 상태는 `IN_PROGRESS`, `SUCCESS`와 `FAILED`입니다.

In [12]:
START_TIME_FOR_INFERENCE_EXECUTIONS = None
END_TIME_FOR_INFERENCE_EXECUTIONS = None
EXECUTION_STATUS = None

execution_summaries = []

while len(execution_summaries) == 0:
    execution_summaries = scheduler.list_inference_executions(
        start_time=START_TIME_FOR_INFERENCE_EXECUTIONS,
        end_time=END_TIME_FOR_INFERENCE_EXECUTIONS,
        execution_status=EXECUTION_STATUS
    )
    if len(execution_summaries) == 0:
        print('WAITING FOR THE FIRST INFERENCE EXECUTION')
        time.sleep(60)
        
    else:
        print('FIRST INFERENCE EXECUTED\n')
        break
            
# execution_summaries

WAITING FOR THE FIRST INFERENCE EXECUTION
WAITING FOR THE FIRST INFERENCE EXECUTION
FIRST INFERENCE EXECUTED



스케줄러를 5분마다 실행하도록 구성했습니다. 몇 분 후 콘솔에서 첫 번째 실행 결과가 입력된 기록을 살펴볼 수 있습니다. 

![Inference history](../assets/schedule_inference_history.png)

스케줄러가 시작될 때, 예를 들어 `datetime.datetime (2021, 1, 27, 9, 15)`일 때 입력 위치에서 **단일** CSV 파일을 찾습니다. 여기에는 타임스탬프가 포함된 파일 명이, 말하자면 다음과 같은 파일 명이 존재해야 합니다.

* subsystem-01_2021012709**10**00.csv가 검색되고 수집됩니다.
* subsystem-01_2021012709**15**00.csv는 수집되지 **않습니다** (다음 추론 실행 시 수집됨).

`subsystem-01_20210127091000.csv` 파일을 연 다음 추론 실행의 DataStartTime과 DataEndTime 사이에 존재하는 시간 행을 찾습니다. 그러한 행을 찾지 못하면 내부 예외를 발생시킵니다.

### 실제 예측 결과 얻기

추론에 성공하면 CSV 파일이 버킷의 출력 위치에 저장됩니다. 각 추론은 `results.csv` 단일 파일이 존재하는 새 폴더를 만듭니다. 해당 파일을 읽고 여기에 내용을 표시해 보겠습니다. 

In [13]:
results_df = scheduler.get_predictions()
results_df.to_csv(os.path.join(INFER_DATA, 'output', 'results.csv'))
results_df.head()

,Predictions
Timestamp,
2021-04-03 06:00:00,0


## 추론 스케줄러 운영
---
### 추론 스케줄러 중단하기
**근검 절약해야합니다**. 스케줄러 실행이 Amazon Lookout for Equipment 비용의 주된 원인입니다. 다음 API를 이용하여 현재 실행 중인 추론 스케줄러를 중지시키세요. 그렇게 하면 주기적인 추론 실행이 중지됩니다.

In [14]:
scheduler.stop()

===== Polling Inference Scheduler Status =====

Scheduler Status: STOPPING
Scheduler Status: STOPPED

===== End of Polling Inference Scheduler Status =====


### 추론 스케줄러 시작하기
다음 API를 사용하여 `STOPPED` 추론 스케줄러를 재시작할 수 있습니다. 

In [15]:
scheduler.start()

===== Polling Inference Scheduler Status =====

Scheduler Status: PENDING
Scheduler Status: RUNNING

===== End of Polling Inference Scheduler Status =====


### 추론 스케줄러 삭제하기
더 이상 사용하지 않는, **중지된** 스케줄러를 삭제할 수 있습니다. 모델 당 하나의 스케줄러만 가질 수 있습니다. 

In [16]:
scheduler.stop()
scheduler.delete()

===== Polling Inference Scheduler Status =====

Scheduler Status: STOPPING
Scheduler Status: STOPPED

===== End of Polling Inference Scheduler Status =====


{'ResponseMetadata': {'RequestId': 'a639de53-6be0-4b43-a3ca-084c8532c214',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a639de53-6be0-4b43-a3ca-084c8532c214',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '0',
   'date': 'Sat, 03 Apr 2021 06:06:53 GMT'},
  'RetryAttempts': 0}}

## 결론
---
이 노트북에서는 노트북 시리즈 파트 3에서 만든 모델을 사용하여 스케줄러를 구성하고 몇 차례 추론을 실행한 다음 예측값을 얻었습니다.